# Exercise: SIMD Data Dependency

Consider the following loop involving four vectors `a`,`b`,`c`, and `d`:

In [3]:
const LOOP_ITERATIONS = 8192
const N = LOOP_ITERATIONS + 2

"naive loop"
function loop_naive!(a, b, c, d)
    @inbounds for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
        b[i+2] = c[i] + d[i]
    end
end

a = rand(Float32, N)
b = rand(Float32, N)
c = rand(Float32, N)
d = rand(Float32, N)

loop_naive!(a,b,c,d)

This loop is hard to auto-vectorize because it has a **data-dependency**: we're reading and writing elements of the vector `b`.

**Task 1**: Check the native code produced for `loop_naive!(a,b,c,d)` and convince yourself that the Julia compiler hasn't vectorized this code. (There shouldn't be any usage of `ymm` or `zmm` registers etc.)

In [4]:
# TODO
@code_native loop_naive!(a,b,c,d)

	.text
	.file	"loop_naive!"
	.globl	"japi1_loop_naive!_762"         # -- Begin function japi1_loop_naive!_762
	.p2align	4, 0x90
	.type	"japi1_loop_naive!_762",@function
"japi1_loop_naive!_762":                # @"japi1_loop_naive!_762"
; ┌ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:5 within `loop_naive!`
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	mov	qword ptr [rbp - 56], rsi
	mov	rax, qword ptr [rsi]
	mov	rcx, qword ptr [rsi + 8]
	mov	rdx, qword ptr [rsi + 16]
	mov	rsi, qword ptr [rsi + 24]
	mov	rax, qword ptr [rax]
	mov	rcx, qword ptr [rcx]
	mov	rdx, qword ptr [rdx]
	mov	rsi, qword ptr [rsi]
; │ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:6 within `loop_naive!`
	lea	rdi, [rax + 32768]
	lea	rbx, [rcx + 32776]
	lea	r8, [rdx + 32768]
	lea	r10, [rsi + 32768]
	cmp	rax, rbx
	setb	r13b
	cmp	rcx, rdi
	setb	r14b
	cmp	rax, r8
	setb	r11b
	cmp	rdx, rdi
	se

*rdi + 24] # xmm1 = mem[0],zero
; │└
; │┌ @ float.jl:409 within `+`
	vaddps	xmm0, xmm0, xmm1
; │└
; │┌ @ array.jl:1021 within `setindex!`
	vmovlps	qword ptr [rcx + 4*rdi + 32], xmm0
	add	rdi, 8
	cmp	rdi, 8192
	jne	.LBB0_8
	jmp	.LBB0_9
.LBB0_5:                                # %L2.preheader
	xor	edi, edi
	.p2align	4, 0x90
.LBB0_6:                                # %L2
                                        # =>This Inner Loop Header: Depth=1
; │└
; │ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:7 within `loop_naive!`
; │┌ @ essentials.jl:13 within `getindex`
	vmovss	xmm0, dword ptr [rax + 4*rdi]   # xmm0 = mem[0],zero,zero,zero
; │└
; │┌ @ float.jl:409 within `+`
	vaddss	xmm0, xmm0, dword ptr [rcx + 4*rdi]
; │└
; │┌ @ array.jl:1021 within `setindex!`
	vmovss	dword ptr [rax + 4*rdi], xmm0
; │└
; │ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:8 within `loop_naive!`
; │┌ @ essentials.jl:13 within `getindex`
	vmovss	xmm0, dword ptr


**Task 2**: Implement the same loop in `loop_naive_simd!` and try to force SIMD-vectorization with the corresponding performance macro. (You shall keep the `@inbounds` as well.)

In [8]:
"naive loop + try force SIMD"
function loop_naive_simd!(a, b, c, d)
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
        b[i+2] = c[i] + d[i]
    end
end

loop_naive_simd!

**Task 3**: Check the native code of `loop_naive_simd!`. Has the code improved? The learning here is that just putting `@simd` in front of a loop and hoping for the best isn't a particularly good strategy 😉

In [9]:
@code_native loop_naive_simd!(a,b,c,d)

	.text
	.file	"loop_naive_simd!"
	.globl	"japi1_loop_naive_simd!_992"    # -- Begin function japi1_loop_naive_simd!_992
	.p2align	4, 0x90
	.type	"japi1_loop_naive_simd!_992",@function
"japi1_loop_naive_simd!_992":           # @"japi1_loop_naive_simd!_992"
; ┌ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:2 within `loop_naive_simd!`
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	mov	qword ptr [rbp - 56], rsi
	mov	rax, qword ptr [rsi]
	mov	rcx, qword ptr [rsi + 8]
	mov	rdx, qword ptr [rsi + 16]
	mov	rsi, qword ptr [rsi + 24]
	mov	rax, qword ptr [rax]
	mov	rcx, qword ptr [rcx]
	mov	rdx, qword ptr [rdx]
	mov	rsi, qword ptr [rsi]
; │ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:3 within `loop_naive_simd!`
; │┌ @ simdloop.jl:75 within `macro expansion`
	lea	rdi, [rax + 32768]
	lea	rbx, [rcx + 32776]
	lea	r8, [rdx + 32768]
	lea	r10, [rsi + 32768]
	cmp	rax, rbx



; ││┌ @ essentials.jl:13 within `getindex`
	vmovsd	xmm0, qword ptr [rdx + 4*rdi + 24] # xmm0 = mem[0],zero
	vmovsd	xmm1, qword ptr [rsi + 4*rdi + 24] # xmm1 = mem[0],zero
; ││└
; ││┌ @ float.jl:409 within `+`
	vaddps	xmm0, xmm0, xmm1
; ││└
; ││┌ @ array.jl:1021 within `setindex!`
	vmovlps	qword ptr [rcx + 4*rdi + 32], xmm0
	add	rdi, 8
; ││└
; ││ @ simdloop.jl:78 within `macro expansion`
; ││┌ @ int.jl:87 within `+`
	cmp	rdi, 8192
	jne	.LBB0_8
	jmp	.LBB0_9
.LBB0_5:                                # %L7.preheader
	xor	edi, edi
	.p2align	4, 0x90
.LBB0_6:                                # %L7
                                        # =>This Inner Loop Header: Depth=1
; ││└
; ││ @ simdloop.jl:77 within `macro expansion` @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:4
; ││┌ @ essentials.jl:13 within `getindex`
	vmovss	xmm0, dword ptr [rax + 4*rdi]   # xmm0 = mem[0],zero,zero,zero
; ││└
; ││┌ @ float.jl:409 within `+`
	vaddss	xmm0, xmm0, dword ptr [rcx + 4*rdi]
; │

**Task 4**: Benchmark and compare the variants. What do you observe?


In [10]:
using BenchmarkTools

@btime loop_naive!($a,$b,$c,$d)
@btime loop_naive_simd!($a,$b,$c,$d)

  2.719 μs (0 allocations: 0 bytes)


  2.814 μs (0 allocations: 0 bytes)



**Task 5**: Take a closer look at the loop. Can you "resolve" the data-dependency issue by splitting up the loop into two separate loops? Implement this improved version in the functions below. Use `@simd` for the loops in the second function. (Again, keep `@inbounds` for all loops in both functions.)

In [11]:
"optimized loop"
function loop_opt!(a, b, c, d)

    @inbounds for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
    end

    @inbounds for i in 1:LOOP_ITERATIONS
        b[i+2] = c[i] + d[i]
    end

end

"optimized loop + `@simd`"

function loop_opt_simd!(a, b, c, d)
    @inbounds @simd for i in 1:LOOP_ITERATIONS
        a[i] = a[i] + b[i]
    end

    @inbounds @simd for i in 1:LOOP_ITERATIONS
        b[i+2] = c[i] + d[i]
    end
    
end

loop_opt_simd! (generic function with 1 method)

**Task 6**: Benchmark those new variants as well.
  * How do they compare to each other?
  * Did the SIMD performance macro help? (Hint: It shouldn't.)
  * How does the performance compare to the unoptimized variants above?

In [12]:
@btime loop_opt!($a,$b,$c,$d)
@btime loop_opt_simd!($a,$b,$c,$d)

  1.399 μs (0 allocations: 0 bytes)


  1.404 μs (0 allocations: 0 bytes)



**Task 7**: Check the native code of e.g. `loop_opt_simd!`. Did it vectorize properly? (Look e.g. for `ymm` and `zmm` registers as well as a block of `vaddps` instructions. Note though, that this is system-dependent.)

In [14]:
@code_native loop_opt_simd!(a,b,c,d)

	.text
	.file	"loop_opt_simd!"
	.globl	"japi1_loop_opt_simd!_1232"     # -- Begin function japi1_loop_opt_simd!_1232
	.p2align	4, 0x90
	.type	"japi1_loop_opt_simd!_1232",@function
"japi1_loop_opt_simd!_1232":            # @"japi1_loop_opt_simd!_1232"
; ┌ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:16 within `loop_opt_simd!`
# %bb.0:                                # %top
	push	rbp
	mov	rbp, rsp
	mov	qword ptr [rbp - 8], rsi
	mov	rax, qword ptr [rsi]
	mov	rdi, qword ptr [rsi + 8]
	mov	rcx, qword ptr [rsi + 16]
	mov	rdx, qword ptr [rsi + 24]
	mov	rsi, qword ptr [rax]
	mov	rax, qword ptr [rdi]
; │ @ /home/javier/JuliaUCL24/exercises/Day2/2_SIMD_datadep/simd_datadep.ipynb:17 within `loop_opt_simd!`
; │┌ @ simdloop.jl:75 within `macro expansion`
	lea	rdi, [rax + 32768]
	cmp	rsi, rdi
	jae	.LBB0_2
# %bb.1:                                # %top
	lea	rdi, [rsi + 32768]
	cmp	rax, rdi
	jae	.LBB0_2
# %bb.8:                                # %L7.preheader
	xor	edi, edi


ymm3, ymm3, ymmword ptr [rax + 4*rdi + 96]
; ││└
; ││┌ @ array.jl:1021 within `setindex!`
	vmovups	ymmword ptr [rsi + 4*rdi], ymm0
	vmovups	ymmword ptr [rsi + 4*rdi + 32], ymm1
	vmovups	ymmword ptr [rsi + 4*rdi + 64], ymm2
	vmovups	ymmword ptr [rsi + 4*rdi + 96], ymm3
; ││└
; ││┌ @ essentials.jl:13 within `getindex`
	vmovups	ymm0, ymmword ptr [rsi + 4*rdi + 128]
	vmovups	ymm1, ymmword ptr [rsi + 4*rdi + 160]
	vmovups	ymm2, ymmword ptr [rsi + 4*rdi + 192]
	vmovups	ymm3, ymmword ptr [rsi + 4*rdi + 224]
; ││└
; ││┌ @ float.jl:409 within `+`
	vaddps	ymm0, ymm0, ymmword ptr [rax + 4*rdi + 128]
	vaddps	ymm1, ymm1, ymmword ptr [rax + 4*rdi + 160]
	vaddps	ymm2, ymm2, ymmword ptr [rax + 4*rdi + 192]
	vaddps	ymm3, ymm3, ymmword ptr [rax + 4*rdi + 224]
; ││└
; ││┌ @ array.jl:1021 within `setindex!`
	vmovups	ymmword ptr [rsi + 4*rdi + 128], ymm0
	vmovups	ymmword ptr [rsi + 4*rdi + 160], ymm1
	vmovups	ymmword ptr [rsi + 4*rdi + 192], ymm2
	vmovups	ymmword ptr [rsi + 4*rdi + 224], ymm3
; ││└
; ││ @ 


; ││└
; ││┌ @ float.jl:409 within `+`
	vaddss	xmm0, xmm0, dword ptr [rdx + 4*rsi]
; ││└
; ││┌ @ array.jl:1021 within `setindex!`
	vmovss	dword ptr [rax + 4*rsi + 8], xmm0
; ││└
; ││┌ @ essentials.jl:13 within `getindex`
	vmovss	xmm0, dword ptr [rcx + 4*rsi + 4] # xmm0 = mem[0],zero,zero,zero
; ││└
; ││┌ @ float.jl:409 within `+`
	vaddss	xmm0, xmm0, dword ptr [rdx + 4*rsi + 4]
; ││└
; ││┌ @ array.jl:1021 within `setindex!`
	vmovss	dword ptr [rax + 4*rsi + 12], xmm0
; ││└
; ││┌ @ essentials.jl:13 within `getindex`
	vmovss	xmm0, dword ptr [rcx + 4*rsi + 8] # xmm0 = mem[0],zero,zero,zero
; ││└
; ││┌ @ float.jl:409 within `+`
	vaddss	xmm0, xmm0, dword ptr [rdx + 4*rsi + 8]
; ││└
; ││┌ @ array.jl:1021 within `setindex!`
	vmovss	dword ptr [rax + 4*rsi + 16], xmm0
; ││└
; ││┌ @ essentials.jl:13 within `getindex`
	vmovss	xmm0, dword ptr [rcx + 4*rsi + 12] # xmm0 = mem[0],zero,zero,zero
; ││└
; ││┌ @ float.jl:409 within `+`
	vaddss	xmm0, xmm0, dword ptr [rdx + 4*rsi + 12]
; ││└
; ││┌ @ array.jl